In [1]:
import os, json
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

from langsmith import Client
client = Client()

In [2]:
from langchain.memory import CosmosDBChatMessageHistory

# Get the conversation for a specific user and session.
session_id = "Agent-Test-Session660"
user_id = "Agent-Test-User127"

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id=session_id,
    user_id=user_id,
)

# Prepare the CosmosDB for the session.
cosmos.prepare_cosmos()

cosmos.messages

[HumanMessage(content='hello how can you help me?'),
 AIMessage(content="Hello! I'm here to assist you with your order at Contoso Coffee. We have a variety of items on our menu including coffees, teas, bakery items, sandwiches, and smoothies. If you have any questions about our menu items or if you're ready to place an order, please let me know."),
 HumanMessage(content='I am in mood for french. What french items do you have?'),
 AIMessage(content='We have two French items on our menu. The first one is Cafe au Lait, a coffee drink made with strong hot coffee and scalded milk, priced at $3.5. The second one is Pain au Chocolat, a classic French pastry filled with chocolate, priced at $3.5.'),
 HumanMessage(content='Can I order 2 each of the first 2 items'),
 AIMessage(content='Of course, you can order two each of Cafe au Lait and Pain au Chocolat. That would be $3.5 (Cafe au Lait) * 2 + $3.5 (Pain au Chocolat) * 2. Do you have any additional notes for these items?'),
 HumanMessage(conte

In [3]:
def cosmos_messages_to_string(messages: list) -> str:
    """
    Converts a list of messages to a string.
    """
    chat_str = ""
    for message in messages:
        chat_str += "Customer: " + message.content + "\n"
        chat_str += "AI: " + message.content + "\n"
    return chat_str

chat_str = cosmos_messages_to_string(cosmos.messages)
print(chat_str)

Customer: hello how can you help me?
AI: hello how can you help me?
Customer: Hello! I'm here to assist you with your order at Contoso Coffee. We have a variety of items on our menu including coffees, teas, bakery items, sandwiches, and smoothies. If you have any questions about our menu items or if you're ready to place an order, please let me know.
AI: Hello! I'm here to assist you with your order at Contoso Coffee. We have a variety of items on our menu including coffees, teas, bakery items, sandwiches, and smoothies. If you have any questions about our menu items or if you're ready to place an order, please let me know.
Customer: I am in mood for french. What french items do you have?
AI: I am in mood for french. What french items do you have?
Customer: We have two French items on our menu. The first one is Cafe au Lait, a coffee drink made with strong hot coffee and scalded milk, priced at $3.5. The second one is Pain au Chocolat, a classic French pastry filled with chocolate, pri

In [4]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, root_validator, ValidationError
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

class OrderedItem(BaseModel):
    """
    Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.
    """
    item: str = Field(description="Name of the item ordered")
    quantity: int = Field(description="Quantity of that ordered item")
    price: float = Field(description="Price of that ordered item")
    item_notes: Optional[str] = Field(description="Any additional notes about that item provided by the customer")
    
class OrderInfo(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract order details such item name, quantity, price, and any additional notes for each item.
    """
    order: List[OrderedItem] = Field(description="List of items ordered and their details")

order_schema = convert_pydantic_to_openai_function(OrderInfo)
print(json.dumps(order_schema, indent=4))

{
    "name": "OrderInfo",
    "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract order details such item name, quantity, price, and any additional notes for each item.",
    "parameters": {
        "title": "OrderInfo",
        "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract order details such item name, quantity, price, and any additional notes for each item.",
        "type": "object",
        "properties": {
            "order": {
                "title": "Order",
                "description": "List of items ordered and their details",
                "type": "array",
                "items": {
                    "title": "OrderedItem",
                    "description": "Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.",


In [5]:
class CustomerName(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract the customer's name.
    """
    customer_name: str = Field(description="Name of the customer")
    
customer_name_schema = convert_pydantic_to_openai_function(CustomerName)
print(json.dumps(customer_name_schema, indent=4))

{
    "name": "CustomerName",
    "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract the customer's name.",
    "parameters": {
        "title": "CustomerName",
        "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract the customer's name.",
        "type": "object",
        "properties": {
            "customer_name": {
                "title": "Customer Name",
                "description": "Name of the customer",
                "type": "string"
            }
        },
        "required": [
            "customer_name"
        ]
    }
}


In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

model = AzureChatOpenAI(temperature=0,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

extraction_functions = [convert_pydantic_to_openai_function(OrderInfo)]

order_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract and save the pertinent details, specifically the ordered food items, quantity ordered, their prices and any additional notes requested for each ordered item included in the order. 
            Make sure to extract only those items that were ordered.
        """),
    ("user", "{input}")
])

extraction_model = model.bind(
    functions=extraction_functions,
    function_call={"name": "OrderInfo"}
)

order_extraction_chain = order_prompt | extraction_model | JsonOutputFunctionsParser()
order_result = order_extraction_chain.invoke({"input": chat_str})
print(json.dumps(order_result, indent=4))


{
    "order": [
        {
            "item": "Cafe au Lait",
            "quantity": 2,
            "price": 3.5,
            "item_notes": "Very hot, less sugar"
        },
        {
            "item": "Pain au Chocolat",
            "quantity": 2,
            "price": 3.5,
            "item_notes": ""
        },
        {
            "item": "Mocha",
            "quantity": 1,
            "price": 4.5,
            "item_notes": ""
        },
        {
            "item": "Lemon Cake",
            "quantity": 1,
            "price": 4.0,
            "item_notes": ""
        },
        {
            "item": "Blueberry Smoothie",
            "quantity": 1,
            "price": 5.5,
            "item_notes": ""
        },
        {
            "item": "Chicken Sandwich",
            "quantity": 1,
            "price": 5.5,
            "item_notes": ""
        }
    ]
}


In [11]:
extraction_functions = [convert_pydantic_to_openai_function(CustomerName)]

name_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract the customer's name included in the order.
        """),
    ("user", "{input}")
])

extraction_model = model.bind(
    functions=extraction_functions,
    function_call={"name": "CustomerName"}
)

name_extraction_chain = name_prompt | extraction_model | JsonOutputFunctionsParser()
name = name_extraction_chain.invoke({"input": chat_str})
customer_name = name["customer_name"]
print(f"Customer name: {customer_name}")

Customer name: Shiva
